# Polars

In [2]:
import polars as pl
import os

path_data = os.path.join(os.getcwd(), 'data') 

df = pl.read_csv(path_data + '/marketing_data.csv')
df = df.with_columns(pl.col("Dt_Customer").str.strptime(pl.Date, "%m/%d/%y"))
df = df.rename({col: col.replace(' ', '').replace('_', '') for col in df.columns})
df = df.with_columns(
    pl.col("Income")
    .str.replace_all(r'[$,]', '')
    .str.replace_all(' ', '')
    .cast(pl.Float64)
    .fill_null(0)
)
df.head()

ID,YearBirth,Education,MaritalStatus,Income,Kidhome,Teenhome,DtCustomer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
i64,i64,str,str,f64,i64,i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
1826,1970,"""Graduation""","""Divorced""",84835.0,0,0,2014-06-16,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,"""SP"""
1,1961,"""Graduation""","""Single""",57091.0,0,0,2014-06-15,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,0,1,1,0,"""CA"""
10476,1958,"""Graduation""","""Married""",67267.0,0,1,2014-05-13,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,0,0,0,0,"""US"""
1386,1967,"""Graduation""","""Together""",32474.0,1,1,2014-05-11,0,10,0,1,0,0,0,1,1,0,2,7,0,0,0,0,0,0,0,"""AUS"""
5371,1989,"""Graduation""","""Single""",21474.0,1,0,2014-04-08,0,6,16,24,11,0,34,2,3,1,2,7,1,0,0,0,0,1,0,"""SP"""


In [22]:
import gcsfs
import os

credentials = 'secretos/credentials.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials

fs = gcsfs.GCSFileSystem()
destination = "gs://data-hist-cf/dlt/deltamkt.parquet"
with fs.open(destination, 'wb') as f:
    df.write_parquet(f)

C:\Users\flavi\AppData\Local\Temp\ipykernel_14224\51357722.py:10: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  df.write_parquet(f)
